# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import requests
import os
from bokeh.models import HoverTool

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/nels.jacobson2/Desktop/Analytics_Class_Folder/040323_Challenge/python-api-challenge/Starter_Code_April_3/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sabang,5.8933,95.3214,84.56,77,87,10.02,ID,1682951536
1,1,port keats,-14.2500,129.5500,75.74,55,67,7.49,AU,1682951537
2,2,kavieng,-2.5744,150.7967,82.02,80,100,10.63,PG,1682951538
3,3,preobrazheniye,42.9019,133.9064,41.40,78,7,4.61,RU,1682951274
4,4,vao,-22.6667,167.4833,75.96,75,100,14.67,NC,1682951539


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
x='Lng',
y='Lat',
c='Humidity',
cmap='YlOrRd',
colorbar=True,
hover_cols=['City'],
size=city_data_df['Humidity']*10,
title='City Humidity',
xlabel='Longitude',
ylabel='Latitude',
xlim=(city_data_df['Lng'].min() - 10, city_data_df['Lng'].max() + 10),
ylim=(city_data_df['Lat'].min() - 10, city_data_df['Lat'].max() + 10),
)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity,_size,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[
    (city_data_df["Max Temp"] >= 70)
    & (city_data_df["Max Temp"] <= 80)
    & (city_data_df["Wind Speed"] < 10)
    & (city_data_df["Cloudiness"] == 0)
].dropna()

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,castro,-24.7911,-50.0119,70.93,56,0,2.93,BR,1682951543
89,89,belle glade,26.6845,-80.6676,78.03,57,0,9.64,US,1682951422
100,100,ewa beach,21.3156,-158.0072,74.43,88,0,4.61,US,1682951626
137,137,niceville,30.5169,-86.4822,71.10,62,0,9.22,US,1682951585
149,149,los llanos de aridane,28.6585,-17.9182,76.75,48,0,7.00,ES,1682951670


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city,
# country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel
# found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,castro,BR,-24.7911,-50.0119,56,
89,belle glade,US,26.6845,-80.6676,57,
100,ewa beach,US,21.3156,-158.0072,88,
137,niceville,US,30.5169,-86.4822,62,
149,los llanos de aridane,ES,28.6585,-17.9182,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "bias": "",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Set the parameters for the API request
    params["filter"] = f"circle:{lng},{lat},10000"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
castro - nearest hotel: Hotel Central Palace
belle glade - nearest hotel: No hotel found
ewa beach - nearest hotel: Hampton Inn & Suites Oahu/Kapolei
niceville - nearest hotel: Holiday Inn Express
los llanos de aridane - nearest hotel: Valle Aridane
tres coracoes - nearest hotel: Hotel Avenida
rio branco do sul - nearest hotel: No hotel found
slidell - nearest hotel: TownePlace Suites Slidell
saint-pierre - nearest hotel: Tropic Hotel
turayf - nearest hotel: المودة


,City,Country,Lat,Lng,Humidity,Hotel Name
8,castro,BR,-24.7911,-50.0119,56,Hotel Central Palace
89,belle glade,US,26.6845,-80.6676,57,No hotel found
100,ewa beach,US,21.3156,-158.0072,88,Hampton Inn & Suites Oahu/Kapolei
137,niceville,US,30.5169,-86.4822,62,Holiday Inn Express
149,los llanos de aridane,ES,28.6585,-17.9182,48,Valle Aridane
202,tres coracoes,BR,-21.6969,-45.2533,59,Hotel Avenida
361,rio branco do sul,BR,-25.1900,-49.3142,68,No hotel found
411,slidell,US,30.2752,-89.7812,56,TownePlace Suites Slidell
486,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
492,turayf,SA,31.6725,38.6637,8,المودة


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Concatenate "Hotel Name" and "Country" columns to create a new column with hotel and country information
hotel_df['Hotel and Country'] = hotel_df['Hotel Name'] + ', ' + hotel_df['Country']

# Configure the map plot
map_plot = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='YlOrRd',
    colorbar=True,
    hover_cols=['City', 'Hotel and Country'],
    size=hotel_df['Humidity']*10,
    title='City Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    xlim=(hotel_df['Lng'].min() - 10, hotel_df['Lng'].max() + 10),
    ylim=(hotel_df['Lat'].min() - 10, hotel_df['Lat'].max() + 10),
)

# Display the map
map_plot


:Scatter   [Lng]   (Lat,Humidity,_size,City,Hotel and Country)